In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from time import sleep, time
from random import uniform

In [2]:
input_file = "input.txt"

if input_file:
    # read keywords from file
    keywords = []
    with open(input_file, "r") as f:
        for line in f:
            tokens = line.split('*')
            if len(tokens) == 3:
                keywords.append(tokens[1])
else:
    # use default keywords
    keywords = ['subreddit', 'wikipedia']    
                
print(keywords)
print(len(keywords))

['dual-pivot', 'fine-tuning pair', 'shared structure', 'out-of-the-box', 'post-training', 'pivot pair', 'completely unsupervised', 'partially unsupervised', 'fourfold', 'light', 'lensing module', 'synthetic bitext', 'bootstrapped bitext', 'unnormalized score', 'degenerate', 'degraded', 'summand', 'summands', 'Arabic-English', 'German-English', 'hubness', 'hub', 'trainable components', 'to some extent', 'resource-rich', 'resource-poor', 'high-resource', 'low-resource', 'lexical constrained decoding', 'left-to-right', 'translation prefix', 'text-infilling', 'text infilling MLM', 'text infilling masked', 'hard constraints', 'maximum a posteriori', 'decoding time', 'multiple iterations', 'post-edited', 'given prefix', 'given suffix', 'burden', 'terminology constraints', 'nested NER', 'non-crossing', 'nested', 'tree fragments', 'chart-based', 'span representation', 'subspan', 'sub-span', 'cursor', 'n-ary', 'bos', 'fencepost', 'fencepost representation', 'deep biaffine function', 'boundary',

In [15]:
chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(options=chrome_options)

# the returned number is unstable, need multiple clicks
clicks = 5

# the wait time needs to be randomized so the scraper is not identified as a bot
# the range is uniformly sampled between [a, b] seconds
# the time also needs to be large enough so the search results could be returned
a = 1
b = 1.5

# the output file, which will be manually edited after obtaining all output
fo = open("output.txt", "a")

# to start from midpoint
start_idx = 353

# with the current parameters, about 20 seconds per keyword
for keyword in keywords[start_idx:]:
    
    start = time()
    
    # random sleep for a few seconds
    sleep(uniform(5, 10))

    url = f"https://aclanthology.org/search/?q={keyword.replace(' ', '+')}"

    # start the driver
    driver.get(url)
    
    nums = []
    
    try:

        # switch between the options multiple times
        for _ in range(clicks):

            # expand the dropdown menu
            buttons = driver.find_elements(by=By.CLASS_NAME, value="gsc-selected-option-container")
            buttons[0].click()

            # choose year of publication
            buttons =  driver.find_elements(by=By.CLASS_NAME, value="gsc-option-menu-item")
            buttons[1].click()

            # some time is needed for the result to change
            sleep(uniform(a,b))

            soup = BeautifulSoup(driver.page_source, "html.parser")

            # result in the format "About 31 results (0.11 seconds)"
            result = soup.find_all("div", {"class": "gsc-result-info"})

            # get the number
            try:
                num = result[0].string.split(' ')[1].replace(',', '')
            except:
                num = 0
            nums.append(num)

            # expand the dropdown menu
            buttons = driver.find_elements(by=By.CLASS_NAME, value="gsc-selected-option-container")
            buttons[0].click()

            # change back to relevance
            buttons =  driver.find_elements(by=By.CLASS_NAME, value="gsc-option-menu-item")
            buttons[0].click()
            sleep(uniform(a,b))

        end = time()

        print(keyword, min(nums), f"({end-start:.2f} seconds)")

        fo.write(f"{keyword}, {min(nums)}\n")
        fo.flush()
        
    except:
        print(f"The {keywords.index(keyword)}-th keyword {keyword} failed!")
        break

# quit the driver
driver.quit()

# close the file
fo.close()

optimize bias 43 (26.14 seconds)
bias optimization 29 (20.16 seconds)
bias parameter 62 (23.43 seconds)
cheaper 42 (22.57 seconds)
BitFit 3 (21.07 seconds)
